In [1]:

# imports
import os
import sys
import types
import json
import base64

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96
interactivity = ''
is_shiny = False
is_dashboard = False
plotly_connected = True

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = "figure"
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  if plotly_connected:
    pio.renderers.default = "notebook_connected"
  else:
    pio.renderers.default = "notebook"
  for template in pio.templates.keys():
    pio.templates[template].layout.margin = dict(t=30,r=0,b=0,l=0)
except Exception:
  pass

# disable itables paging for dashboards
if is_dashboard:
  try:
    from itables import options
    options.dom = 'fiBrtlp'
    options.maxBytes = 1024 * 1024
    options.language = dict(info = "Showing _TOTAL_ entries")
    options.classes = "display nowrap compact"
    options.paging = False
    options.searching = True
    options.ordering = True
    options.info = True
    options.lengthChange = False
    options.autoWidth = False
    options.responsive = True
    options.keys = True
    options.buttons = []
  except Exception:
    pass
  
  try:
    import altair as alt
    # By default, dashboards will have container sized
    # vega visualizations which allows them to flow reasonably
    theme_sentinel = '_quarto-dashboard-internal'
    def make_theme(name):
        nonTheme = alt.themes._plugins[name]    
        def patch_theme(*args, **kwargs):
            existingTheme = nonTheme()
            if 'height' not in existingTheme:
              existingTheme['height'] = 'container'
            if 'width' not in existingTheme:
              existingTheme['width'] = 'container'

            if 'config' not in existingTheme:
              existingTheme['config'] = dict()
            
            # Configure the default font sizes
            title_font_size = 15
            header_font_size = 13
            axis_font_size = 12
            legend_font_size = 12
            mark_font_size = 12
            tooltip = False

            config = existingTheme['config']

            # The Axis
            if 'axis' not in config:
              config['axis'] = dict()
            axis = config['axis']
            if 'labelFontSize' not in axis:
              axis['labelFontSize'] = axis_font_size
            if 'titleFontSize' not in axis:
              axis['titleFontSize'] = axis_font_size  

            # The legend
            if 'legend' not in config:
              config['legend'] = dict()
            legend = config['legend']
            if 'labelFontSize' not in legend:
              legend['labelFontSize'] = legend_font_size
            if 'titleFontSize' not in legend:
              legend['titleFontSize'] = legend_font_size  

            # The header
            if 'header' not in config:
              config['header'] = dict()
            header = config['header']
            if 'labelFontSize' not in header:
              header['labelFontSize'] = header_font_size
            if 'titleFontSize' not in header:
              header['titleFontSize'] = header_font_size    

            # Title
            if 'title' not in config:
              config['title'] = dict()
            title = config['title']
            if 'fontSize' not in title:
              title['fontSize'] = title_font_size

            # Marks
            if 'mark' not in config:
              config['mark'] = dict()
            mark = config['mark']
            if 'fontSize' not in mark:
              mark['fontSize'] = mark_font_size

            # Mark tooltips
            if tooltip and 'tooltip' not in mark:
              mark['tooltip'] = dict(content="encoding")

            return existingTheme
            
        return patch_theme

    # We can only do this once per session
    if theme_sentinel not in alt.themes.names():
      for name in alt.themes.names():
        alt.themes.register(name, make_theme(name))
      
      # register a sentinel theme so we only do this once
      alt.themes.register(theme_sentinel, make_theme('default'))
      alt.themes.enable('default')

  except Exception:
    pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass

# interactivity
if interactivity:
  from IPython.core.interactiveshell import InteractiveShell
  InteractiveShell.ast_node_interactivity = interactivity

# NOTE: the kernel_deps code is repeated in the cleanup.py file
# (we can't easily share this code b/c of the way it is run).
# If you edit this code also edit the same code in cleanup.py!

# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
run_path = 'L1VzZXJzL2NoZXJ5bGtvdWFkaW8vRG9jdW1lbnRzL1JlcG9zaXRvcmllcy9wZXJzb25hbC13ZWJzaXRlLzNBL3Byb2Nfc3RvY2hhc3RpcXVl'
if run_path:
  # hex-decode the path
  run_path = base64.b64decode(run_path.encode("utf-8")).decode("utf-8")
  os.chdir(run_path)

# reset state
%reset

# shiny
# Checking for shiny by using False directly because we're after the %reset. We don't want
# to set a variable that stays in global scope.
if False:
  try:
    import htmltools as _htmltools
    import ast as _ast

    _htmltools.html_dependency_render_mode = "json"

    # This decorator will be added to all function definitions
    def _display_if_has_repr_html(x):
      try:
        # IPython 7.14 preferred import
        from IPython.display import display, HTML
      except:
        from IPython.core.display import display, HTML

      if hasattr(x, '_repr_html_'):
        display(HTML(x._repr_html_()))
      return x

    # ideally we would undo the call to ast_transformers.append
    # at the end of this block whenver an error occurs, we do 
    # this for now as it will only be a problem if the user 
    # switches from shiny to not-shiny mode (and even then likely
    # won't matter)
    import builtins
    builtins._display_if_has_repr_html = _display_if_has_repr_html

    class _FunctionDefReprHtml(_ast.NodeTransformer):
      def visit_FunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

      def visit_AsyncFunctionDef(self, node):
        node.decorator_list.insert(
          0,
          _ast.Name(id="_display_if_has_repr_html", ctx=_ast.Load())
        )
        return node

    ip = get_ipython()
    ip.ast_transformers.append(_FunctionDefReprHtml())

  except:
    pass

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v

  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define
globals()["__spec__"] = None

{"/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/importlib/_bootstrap.py": 1731323902.0, "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/importlib/_bootstrap_external.py": 1731323902.0, "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/codecs.py": 1731323889.0, "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/encodings/aliases.py": 1731323904.0, "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/encodings/cp437.py": 1731323905.0, "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/encodings/__init__.py": 1731323904.0, "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/encodings/utf_8.py": 1731323906.0, "/Library/Developer/CommandLineT

/var/folders/4g/2x7xwt6106l13nxqljfks4gm0000gn/T/ipykernel_31920/2494819089.py:25: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


In [2]:
# Calcul d'un call
from scipy.stats import norm
import numpy as np
from scipy.optimize import minimize

def FormulaBS(S,K,r,tau,sigma):
    d1 = (np.log(S/K) + (r + 0.5 *sigma**2)*tau ) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    call = S*norm.cdf(d1) - K* np.exp( - tau * r) * norm.cdf(d2)
    return call

In [3]:
def FormulaBSMC(K,r,tau,sigma,M, S0) :
    # Simulation du M mouvement brownien de loi N(0, sigma*sqrt(T))
    var_brown = sigma * np.sqrt(tau)
    W_T= np.random.normal(0,var_brown,M)

    S_T = S0 * np.exp((r - 0.5 * sigma**2) * tau + W_T)
    payoff = np.maximum(S_T - K, 0)

    call = np.exp(-r*tau) * np.mean(payoff)
    return call

In [4]:
# Calcul du prix d'un call lors t=O, T=6mois, S0= 42, K=40, r=10%, sigma=20%
S0 = 42
K = 40
r = 0.1
tau = 6
sigma = 0.2

call_BS = FormulaBS(S0,K,r,tau,sigma)
print(f"Le prix d'un call est de maturité {tau}mois et de strike {K} est de {call_BS}")

Le prix d'un call est de maturité 6mois et de strike 40 est de 20.67722481517296


In [5]:
M_values = [500, 5000, 50000]
for M in M_values:
	call_BSMC = FormulaBSMC(K, r, tau, sigma, M, S0)
	print(f"Le prix d'un call avec M={M}, de maturité {tau}mois et de strike {K} est de {call_BSMC}")

Le prix d'un call avec M=500, de maturité 6mois et de strike 40 est de 20.21549073157966
Le prix d'un call avec M=5000, de maturité 6mois et de strike 40 est de 20.341643527763484
Le prix d'un call avec M=50000, de maturité 6mois et de strike 40 est de 20.71029139679338


In [6]:
# Calcul du prix d'un call lors t=O, T=3mois, S0= 42, K=40, r=10%, sigma=20%
tau = 3
call_BS = FormulaBS(S0,K,r,tau,sigma)

print(f"Le prix d'un call est de maturité {tau}mois et de strike {K} est de {call_BS}")

for M in M_values:
    call_BSMC = FormulaBSMC(K, r, tau, sigma, M, S0)
    print(f"Le prix d'un call avec M={M}, de maturité {tau}mois et de strike {K} est de {call_BSMC}")

Le prix d'un call est de maturité 3mois et de strike 40 est de 13.362666146646749
Le prix d'un call avec M=500, de maturité 3mois et de strike 40 est de 13.949252228829925
Le prix d'un call avec M=5000, de maturité 3mois et de strike 40 est de 13.440144185789123
Le prix d'un call avec M=50000, de maturité 3mois et de strike 40 est de 13.383668100223682


In [7]:
def FormulaBSGreeks(S,K,r,tau,sigma):
    d1 = (np.log(S/K) + (r + 0.5 *sigma**2)*tau ) / (sigma * np.sqrt(tau))
    # d2 = d1 - sigma * np.sqrt(tau)

    delta = norm.cdf(d1)
    gamma = (1/(S*sigma*np.sqrt(tau))) * norm.pdf(d1)
    vega = S * np.sqrt(tau) * norm.pdf(d1)

    return delta, gamma, vega

In [8]:
S = 100
K = 110
r = 0.1
tau = 0.5
sigma = 0.2

delta, gamma, vega = FormulaBSGreeks(S=S,K=K,r=r,tau=tau,sigma=sigma)
print(f"Les greeks d'un call sont de maturité {tau}mois et de strike {K} sont delta={delta}, gamma={gamma}, vega={vega}")

Les greeks d'un call sont de maturité 0.5mois et de strike 110 sont delta=0.40141715171302983, gamma=0.027343746144537384, vega=27.343746144537384


In [9]:
# Supposons qu'on a pas la formule des greeks
# comment calculer les greeks
# Approche montecarlo 

def FormulaBSGreeksMC(K,r,tau,sigma,M, S0) :
    d1 = (np.log(S/K) + (r + 0.5 *sigma**2)*tau) / (sigma * np.sqrt(tau))
    # Simulation du M mouvement brownien de loi N(0, sigma*sqrt(T))
    var_brown = sigma * np.sqrt(tau)
    W_T= np.random.normal(0,var_brown,M)

    S_T = S0 * np.exp((r - 0.5 * sigma**2) * tau + W_T)

    delta = np.exp(-r*tau) * np.mean((S_T > K) * S_T / S0)
    gamma = norm.pdf(d1)/(S0 * sigma * np.sqrt(tau))
    vega = np.exp(-r*tau) * np.mean((S_T > K) * (S_T/sigma) *(np.log(S_T/S0) - (r + 0.5 * sigma**2)*tau))

    return delta, gamma, vega

delta, gamma, vega = FormulaBSGreeksMC(K,r,tau,sigma,500, S)
print(f"Les greeks d'un call sont de maturité {tau}mois et de strike {K} sont delta={delta}, gamma={gamma}, vega={vega}")

Les greeks d'un call sont de maturité 0.5mois et de strike 110 sont delta=0.42713711028344337, gamma=0.027343746144537384, vega=27.94419858966199


In [10]:
# Mérhode de différencee finie basé sur la méthode de Taylor
def FormulaBSGreeks_FD_num(S, K, r, tau, sigma, delta_S=1e-5):
    """
    Calcule les Grecs (Delta, Gamma, Vega) pour une option call
    par différences finies.

    Paramètres:
    S : float - Prix actuel de l'actif sous-jacent
    K : float - Prix d'exercice de l'option (strike)
    r : float - Taux d'intérêt sans risque (annualisé)
    tau : float - Temps jusqu'à la maturité (en années)
    sigma : float - Volatilité de l'actif sous-jacent (annualisée)
    epsilon : float - Petit incrément pour les différences finies

    Retour:
    tuple - (Delta, Gamma, Vega)
    """
    # Delta
    delta = (FormulaBS(S + delta_S, K, r, tau, sigma) - FormulaBS(S - delta_S, K, r, tau, sigma)) / (2 * delta_S)

    # Gamma
    gamma = (FormulaBS(S + delta_S, K, r, tau, sigma) - 2 * FormulaBS(S, K, r, tau, sigma) + FormulaBS(S - delta_S, K, r, tau, sigma)) / (delta_S**2)

    # Vega
    vega = (FormulaBS(S, K, r, tau, sigma + delta_S) - FormulaBS(S, K, r, tau, sigma - delta_S)) / (2 * delta_S)

    return delta, gamma, vega

delta, gamma, vega = FormulaBSGreeks_FD_num(S,K,r,tau,sigma)
print(f"Les greeks d'un call sont de maturité {tau}mois et de strike {K} sont delta={delta}, gamma={gamma}, vega={vega}")

Les greeks d'un call sont de maturité 0.5mois et de strike 110 sont delta=0.4014171516075748, gamma=0.027142732506035824, vega=27.34374614092871


In [11]:
def ImpliedVolBS(S,K,r,tau, Call):
    # optim function
    def obj_func(sigma):
        return (Call - FormulaBS(S,K,r,tau,sigma))**2
    
    res = minimize(obj_func, 0.2)
    sigma = res.x[0]
    return sigma


In [12]:
# Calcul de la volatilité d'une option d'achat à la monnaie de maturité 3 mois et de strike K=S=4.58 sachant que S0=100; r=5%

S = 100
K = 100
r = 0.05
tau = 3/12
Call = 4.58
sigma = ImpliedVolBS(S,K,r,tau, Call)

print(f"La volatilité implicite d'un call est de maturité {tau}mois et de strike {K} est de {sigma}")

La volatilité implicite d'un call est de maturité 0.25mois et de strike 100 est de 0.19821832844648876


In [13]:
# Calcul de la volatilité d'une option d'achat à la monnaie de maturité 6 mois et de strike K=S=5.53 sachant que S0=100; r=5%

S = 100
K = 100
r = 0.05
tau = 0.5
Call = 5.53
sigma = ImpliedVolBS(S,K,r,tau, Call)

print(f"La volatilité implicite d'un call est de maturité {tau}mois et de strike {K} est de {sigma}")

La volatilité implicite d'un call est de maturité 0.5mois et de strike 100 est de 0.15010660990708588
